In [1]:
import pandas as pd

In [2]:
executions = pd.read_csv("executions.csv", dtype={"Symbol": "category"},
                         parse_dates=["ExecutionTransactTime", "OrderTransactTime"],
                         date_format=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
                         )
executions.head()

,ClOrdID,ExecutionTransactTime,OrderTransactTime,Symbol,Side,OrderQty,LimitPrice,AvgPx,LastMkt
0,ID3,20250910-04:02:34.713,20250910-04:02:34.713,SMCX,2.0,10.0,32.98,32.98,ID7
1,ID9,20250910-04:06:14.467,20250910-04:06:14.467,CRWV,1.0,5.0,105.79,105.79,ID7
2,ID13,20250910-04:06:27.982,20250910-04:06:27.983,TSLA,2.0,23.0,347.61,347.61,ID7
3,ID17,20250910-04:06:49.075,20250910-04:06:49.075,PLTR,1.0,10.0,164.40,164.40,ID7
4,ID20,20250910-04:07:12.532,20250910-04:07:12.533,SOFI,1.0,100.0,25.91,25.91,ID7


In [ ]:
quotes = pd.read_csv("quotes_2025-09-10_small.csv.gz", 
                     dtype = {"ticker": "category"})

quotes["sip_timestamp"] = pd.to_datetime(quotes["sip_timestamp"], unit="ns")

TypeError: to_datetime() got an unexpected keyword argument 'units'

In [5]:
quotes.dtypes

ticker           category
bid_price         float64
ask_price         float64
bid_size            int64
ask_size            int64
bid_exchange        int64
ask_exchange        int64
sip_timestamp       int64
dtype: object

In [9]:
quotes.head()

,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,A,0.00,0.0,0,0,19,19,NaT
1,A,0.00,0.0,0,0,8,8,NaT
2,A,0.00,0.0,0,0,20,20,NaT
3,A,95.18,0.0,1,0,8,8,NaT
4,A,98.25,0.0,1,0,8,8,NaT
